SSE (Server-Sent Events) is how LLM APIs stream responses token-by-token. Format: `data: {json}\n\n` with `[DONE]` signaling completion.

#### Part 1: Serving with FastAPI

In [ ]:
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
import asyncio, json

app = FastAPI()

class ChatRequest(BaseModel):
    messages: list
    stream: bool = True

async def generate_response(messages: list):
    for token in "Hello! This is streamed.".split():
        yield f"data: {json.dumps({'choices': [{'delta': {'content': token + ' '}}]})}\n\n"
        await asyncio.sleep(0.1)
    yield "data: [DONE]\n\n"

@app.post("/v1/chat/completions")
async def chat(req: ChatRequest):
    return StreamingResponse(generate_response(req.messages), media_type="text/event-stream")

#### Part 2: Consuming with Requests

In [ ]:
import json, requests

def consume_sse(url, headers=None, payload=None):
    resp = requests.post(url, headers=headers, json=payload, stream=True)
    resp.raise_for_status()
    for line in resp.iter_lines():
        if line and (decoded := line.decode('utf-8')).startswith('data: '):
            if (data := decoded[6:]) == '[DONE]': break
            yield json.loads(data)

# Usage: stream from OpenAI-compatible API
def stream_chat(url, key, messages):
    for chunk in consume_sse(url, {"Authorization": f"Bearer {key}"}, {"messages": messages, "stream": True}):
        if content := chunk.get('choices', [{}])[0].get('delta', {}).get('content'):
            print(content, end='', flush=True)